## Objetivo

Diseñar un modelo de datos que facilite el almacenamiento y la consulta de la información analítica, de manera que se pueda predecir la facturación total de cada usuario en un periodo dado.
Se explicará cómo se construye un modelo en estrella (Star Schema) y cómo se relacionan las tablas de dimensiones (Usuarios, Planes, Tiempo) con la tabla de hechos.

## Schema

- Tiene una tabla central de hechos y varias tablas de dimensiones.
- Es más simple y eficiente para consultas analíticas, pues minimiza la cantidad de joins.
- La tabla de hechos va a tener la variable objetivo facturación_total y otros indicadores agregados de consumo, y se relacionará con Dim_Usuarios y Dim_Planes

Tabla de hechos (Fact_Facturacion):
- user_id: Identificador del usuario.
- plan: Identificador o nombre del plan (como llave foránea a Dim_Planes).
- periodo: Periodo de facturación (por ejemplo, extraído de reg_date o definido a partir de la fecha de consumo).
- facturacion: Monto total que paga el usuario (costo base + costos extra).

Dimension de Usuarios (Dim_Usuarios):
- user_id: Llave primaria.
- first_name, last_name, age, city, reg_date, churn_date, etc.

Dimension Planes Dim_Planees:
- plan o plan_name: Llave primaria.
- usd_monthly_pay, minutes_included, messages_included, mb_per_month_included, usd_per_minute, usd_per_message, usd_per_gb, etc.

Dimension Tiempo (Dim_Tiempo):
- periodo: Por ejemplo, mes y año.
- Otros atributos temporales (trimestre, día, etc.), si son relevantes para el análisis.


## Implementacion

In [40]:
import pandas as pd

In [41]:
df = pd.read_csv('../data/ml/df_features.csv')

In [ ]:
#Periodo (mes de reg_date, o podrías usar otra fecha de consumo)
df['periodo'] = pd.to_datetime(df['reg_date']).dt.to_period('M')

# Seleccionar las columnas relevantes para la tabla de hechos
fact_table = df[['user_id', 'plan', 'periodo', 'facturacion', 'total_minutes', 'total_messages', 'total_mb']]

In [43]:
# Guardar la tabla de hechos
fact_table.to_csv('../data/ml/Fact_Facturacion.csv', index=False)

In [ ]:
# Dim_Usuarios
dim_usuarios = df[['user_id', 'first_name', 'last_name', 'age', 'city', 'reg_date', 'churn_date']].drop_duplicates()
dim_usuarios.to_csv('../data/ml/Dim_Usuarios.csv', index=False)

In [ ]:
#Dim_Planes
# Seleccionar columnas únicas de planes
dim_planes = df[['plan', 'usd_monthly_pay', 'minutes_included', 'messages_included',
                 'mb_per_month_included', 'usd_per_minute', 'usd_per_message', 'usd_per_gb']].drop_duplicates()
dim_planes.to_csv('../data/ml/Dim_Planes.csv', index=False)

In [ ]:
#Dim_Tiempo
dim_tiempo = pd.DataFrame(df['periodo'].unique(), columns=['periodo'])
dim_tiempo['mes'] = dim_tiempo['periodo'].astype(str).str.split('-').str[1]
dim_tiempo['año'] = dim_tiempo['periodo'].astype(str).str.split('-').str[0]
dim_tiempo.to_csv('../data/ml/Dim_Tiempo.csv', index=False)

In [47]:
print("Modelo de Datos (Star Schema) generado y guardado en la carpeta 'data/ml'.")

Modelo de Datos (Star Schema) generado y guardado en la carpeta 'data/ml'.
